<a href="https://colab.research.google.com/github/jpsbali/GoogleColab/blob/main/Apple_CLaRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install torch
%pip install -U transformers==4.57.1 trl==0.25.1 datasets==4.4.1

In [2]:
from google.colab import userdata
from huggingface_hub import login

# Login into Hugging Face Hub
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [ ]:
from huggingface_hub import snapshot_download
import os

# Define the model repo ID and the specific subfolder you want
REPO_ID = "apple/CLaRa-7B-Instruct"
SUBFOLDER_NAME = "compression-16"

# Define the local directory where you want to save the files
LOCAL_DIR = "./local_model_folder"

# Create the local directory if it doesn't exist
os.makedirs(LOCAL_DIR, exist_ok=True)

# Download only the files within the specified subfolder
# The 'allow_patterns' argument takes a list of patterns to match files/folders
downloaded_folder_path = snapshot_download(
    repo_id=REPO_ID,
    local_dir=LOCAL_DIR,
    allow_patterns=[f"{SUBFOLDER_NAME}/*"],
    ignore_patterns=["*.safetensors", "*.bin"] # Optional: ignore large files if not needed
)

print(f"Downloaded subfolder located at: {downloaded_folder_path}/{SUBFOLDER_NAME}")

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import os

#Construct the full local path to the 'compression-16' model files
local_model_path = os.path.join('/content/local_model_folder', 'compression-16')

#Load the model and tokenizer from the local path
unirag = AutoModel.from_pretrained(
    local_model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    local_files_only=True).to("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained( local_model_path, trust_remote_code=True, local_files_only=True )

documents = [ [
    "Weldenia is a monotypic genus of flowering plant in the family Commelinaceae...",
    "Hagsatera is a genus of flowering plants from the orchid family...",
    "Alsobia is a genus of flowering plants in the family Gesneriaceae..." ] ]

questions = [ "Which genus of plant grows originally in Mexico and Guatemala, Phylica or Weldenia?" ]

#Instruction-tuned usage
out = unirag.generate_from_text( questions=questions, documents=documents, max_new_tokens=64 )

print("Generated answer:", out)